## Step 1: Installing dependencies and setting up the enviornment

In [0]:
!pip install -q tensorflow-gpu==1.14.0

## Step 2 Import Project Dependencies

In [2]:
import tensorflow as tf 
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'1.14.0'

## Step 3 Dataset Preprocessing

### Loading FashionMNIST dataset

In [0]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### Image normalization

In [0]:
# divide by 255
X_train =  X_train / 255.
X_test = X_test / 255. 

### reshaping datast

In [5]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Step 4 Building the Model

### Defining the model

In [0]:
model = tf.keras.models.Sequential()

### Building the model

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
#output layer. must have same number of units as classes in our dataset
model.add(tf.keras.layers.Dense(units=10, activation = 'softmax'))

W0813 00:51:12.652720 140065937168256 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Compiling the model

In [0]:
model.compile(optimizer ='adam', loss='sparse_categorical_crossentropy', metrics = ['sparse_categorical_accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


###Train the model

In [12]:
model.fit(X_train, y_train, epochs = 5 )

Epoch 1/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.5346 - sparse_categorical_accuracy: 0.8103
Epoch 2/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.3991 - sparse_categorical_accuracy: 0.8550
Epoch 3/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.3685 - sparse_categorical_accuracy: 0.8655
Epoch 4/5
60000/60000 [==============================] - 6s 94us/sample - loss: 0.3456 - sparse_categorical_accuracy: 0.8735
Epoch 5/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.3316 - sparse_categorical_accuracy: 0.8776


### Evaluate the model

In [13]:
test_loss, test_accuracy = model.evaluate(X_test,y_test)
print(f"test accuracy: {test_accuracy}")

10000/10000 [==============================] - 1s 64us/sample - loss: 0.3659 - sparse_categorical_accuracy: 0.8691
test accuracy: 0.8690999746322632


## Step 5 Saving the model 

In [0]:
model_name = "fashion_mobile_model.h5"
tf.keras.models.save_model(model, model_name)

### Creating the TFLite Converter

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

W0813 00:51:48.187191 140065937168256 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 00:51:48.193053 140065937168256 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 00:51:48.751016 140065937168256 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/util.py:238: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is

### Converting the model.

In [0]:
tflite_model = converter.convert()

### Saving the TFLite version of the model

In [0]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)